In [86]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
target_kern_name = 'matrixMulCUDA'
trace_file = 'trace64.csv'
metrics_file = 'metrics64.csv'
sass_result = 'mm8.sm52.sass.result'
    
# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)
    
out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)
    
# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

In [88]:
df_kern

,kern_name,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,shared_mem,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,matrixmulcuda,0.489635,80.0,40.0,1.0,8.0,8.0,1.0,29.0,512.0,...,Mid (4),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.667005,11.992918%,0.000000%


In [89]:
df_kern.to_csv('test_mm8.csv', index=False, encoding='utf-8')

In [90]:
warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)


Total running threads : 204800.0

thread_fp_32: 320.0
thread_fp_64: 0.0
thread_integer: 549.0
thread_bit_convert: 0.0
thread_control: 40.0
thread_compute_ld_st: 561.0
thread_thread_misc: 90.0
instructions per thread : 1560.0
instructions per warp   : 1562.0
ldg (global load) clks (per warp) 	 = 13000.0
stg (global store) clks (per warp) 	 = 19.0
lds (shared load) clks (per warp) 	 =  390.0
sts (shared store) clks (per warp) 	 = 57.0
=> Memory inst. (per warp) 		 = 13466.0 (clocks)

Integer inst. (per warp) 	= 8235.0 (clocks)
FP32 inst. (per warp) 		= 4800.0 (clocks)
FP64 inst. (per warp) 		= 0.0 (clocks)
Compute ldst inst. (per warp) 	= 8415.0 (clocks)
=> Compute inst. (per warp) 	= 21450.0 (clocks)

Compute-intensive Warp
